In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'e4e1b4447e20',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'DNoCuE3RTv2p_fQWsmn1YA',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [2]:
path = '/opt/ml/data_v3/wikipedia_documents.json'

In [3]:
import json #json 수정하려고 열음
with open(path, 'r') as f:
    wiki = json.load(f)

In [4]:
wiki['224']

{'text': '당의 경선중 박근혜 측에서 BBK 의혹을 제기하면서부터 친이(親李, 친이명박) 진영과 친박(親朴, 친박근혜) 진영의 갈등이 시작되었다. 박근혜 측은 결과에 승복해 갈등은 사라지는 듯하다가, 본격적으로 갈등이 심화되어 집단탈당사태가 일어난건 공천파동에서였다. 이후 친박연대가 창당되면서 미디어법 등 몇 가지 정책들에서 친이계와 친박계의 미묘한 의견 차이는 있어왔으나, 2010년 1월 이명박 대통령이 세종시 사수안을 번복하고 제안한 세종시 수정안 방침에 대해 친박연대를 비롯한 당의 주요 친박계 의원들이 반발하면서 그 갈등이 더욱 커지고 있었다. 박근혜는 원안 추진을 재차 강조하여 화합의 가능성이 낮아진 데 이어 심지어 분당 가능성까지 나왔다 이전에 홍준표는 박근혜의 세종시 원안 추진론 발언에 대해 \'탈당\' 이야기까지 나온 상황이었고, 정몽준이 당론을 변경해야 한다는 말에 박근혜가 "국민의 신뢰를 잃은 것에 대해 책임지실 문제"라며 정몽준 책임론을 들고 나왔으며, 정운찬 국무총리를 비롯한 정부 고위직 관계자들까지 박근혜를 압박하는 상황이 벌어지면서 갈등의 골이 깊어지고 있었다. 이러한 갈등 속에서 야당들이 추진했던 정운찬 총리 해임안을 여당 내 친박계 의원들이 찬성한다는 입장을 밝히자 친이계 의원들은 "총리 해임안에 찬성하는 것은 분당하자는 것" 이라며 계파간의 대립이 극화되었다. 6월 29일 세종시와 관련해서 친박계 의원들을 비롯하여 야당의원들이 반대하는 가운데 수정안이 부결되었다. 이 때문에 친이, 친박간의 갈등이 당분간 해소되기 어렵다는 관측들이 나왔다. 하지만 2010년 8월, 이명박 대통령과 박근혜 전 대표가 전격 회동하면서 화해 분위기가 조성되었고 양측은 우호적인 분위기를 유지하다가 2011년에 들어서면서 친이계가 급속히 몰락하면서 당 중심이 친박계로 이동하였다는 관측이 지배적이다.',
 'title': '자유한국당'}

In [5]:
# for i in range(len(wiki)):
#     del wiki[str(i)]['corpus_source']# 필요없는거 삭제
#     del wiki[str(i)]['domain']
#     del wiki[str(i)]['author']
#     del wiki[str(i)]['html']
#     del wiki[str(i)]['document_id']
#     del wiki[str(i)]['url']

In [6]:
wiki['16']['text']

'통계학은 관찰 및 조사로 얻을 수 있는 데이터로부터, 응용 수학의 기법을 이용해 수치상의 성질, 규칙성 또는 불규칙성을 찾아낸다. 통계적 기법은, 실험 계획, 데이터의 요약이나 해석을 실시하는데 있어서의 근거를 제공하는 학문이며, 폭넓은 분야에서 응용되어 실생활에 적용되고 있다 통계학은 실증적인 뿌리를 가지고 있으며 실질적 활용에 초점을 맞추고 있기 때문에, 흔히 순수수학과는 다소 구분되는 응용수학의 일종으로 여겨진다. 통계학의 방법을 통해, 실제의 수치들을 왜곡하여 해석하는 것을 막고 연구를 바탕으로 합리적인 의사결정을 할 수 있다. 통계학은 과학, 산업, 또는 사회의 문제에 적용되며 모집단을 연구하는 과정이 우선시된다. 모집단은 "한나라 안에 사는 모든 사람" 또는 "크리스탈을 구성하는 모든 원자"와 같이 일정한 특성을 지닌 집단이면 어느 것이든 가능하다. 통계학자들은 전체인구(인구조사를 하는 기업)에 대한 데이터를 편집한다. 이것은 정부의 통계관련 법률요약집같은 조직화된 방법으로 수행될 수도 있다. 기술통계학은 모집단의 데이터를 요약하는데 사용된다. 도수 및 비율 (경주 등) 범주 형 데이터를 설명하는 측면에서 더 유용할 동안 수치 기술자는 연속적인 데이터 유형 (소득 등)에 대한 평균과 표준 편차를 포함한다. 데이터 분석 방법 엄청난 자료가 연구되는 현대 사회에서 경제지표연구, 마케팅, 여론조사, 농업, 생명과학, 의료의 임상연구 등 다양한 분야에서 응용되고 있는 통계는 단연 우리 사회에서 가장 필요하고 실용적인 학문이라고 할 수 있다.'

In [7]:
# import re
# for i in range(len(wiki)):
#     temp = wiki[f'{i}']['text']
#     temp = re.sub('[#*~"\"]','',temp)
#     temp = temp.split('\n')
#     temp = [s.strip() for s in temp]
#     temp = list(filter(None, temp))
#     temp = ' '.join(temp)
#     wiki[f'{i}']['text'] = temp

In [8]:
wiki['14']

{'text': '파이의 날(Pi Day)은 원주율을 기념하는 날이다. 파이의 날은 원주율의 근삿값이 3.14이어서 3월 14일에 치러진다. 보통 3.14159에 맞추기 위해 오후 1시 59분에 기념하는데, 오후 1시 59분은 엄밀히 말하면 13시 59분이기 때문에 오전 1시 59분 혹은 15시 9분(오후 3시 9분)에 치러야 한다고 주장하는 사람도 있다. 세계 각국의 수학과에서 기념행사를 연다. 3월 14일은 알베르트 아인슈타인의 생일이면서 스티븐 호킹의 기일이기도 하다. 이 날은 여러 방법으로 기념된다. 사람들이 모여서 원주율이 생활에서 어떤 역할을 했는지 이야기하고 원주율이 없는 세상을 상상해 본다. 모임에서는 보통 파이를 먹는다. 또한 많은 행사에서 원주율을 소수점 아래의 숫자를 얼마나 많이 외우는지 겨루는 대회가 열린다. 분수 3과 7분의 1을 가분수로 나타내면 7분의 22가 되는데, 이를 유리수(순환소수)로 나타내면 3.142857...로 π의 근삿값이 되므로 7월 22일을 파이 근삿값 날로 부르기도 한다.',
 'title': '파이의 날'}

In [9]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type" : "stop",
                        "stopwords_path" : "my_stopwords.txt" # /etc/elastic안에 txt파일이 존재해야 댑니다
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "none",
                        "filter" : ["my_stop_filter"]# 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", # 먼지 잘모르겟
            "properties": {
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "title":{"type": "text"}
                }
            }
        }

In [10]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

<ipython-input-10-c1b8263f83d6>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=index_config)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [11]:
import time
import tqdm

for doc_id, doc in tqdm.tqdm(wiki.items(),total=len(wiki)):
    
    es.index(index=INDEX_NAME, id=doc_id, body=doc)
    # time.sleep(0.1) # 문서집어넣기 time슬립너어야 에러 안걸림 (너무빨리 문서를 호출해서)

  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-11-5a9fb178b7ff>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, id=doc_id, body=doc)
100%|██████████| 60613/60613 [04:37<00:00, 218.40it/s]


In [12]:
import pprint
# doc = es.get(index=INDEX_NAME, id=60000)#
# pprint.pprint(doc)

In [13]:
tv = es.termvectors(index=INDEX_NAME, id=2, body={"fields" : ["content","title"]})

In [14]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 6274,
                                                 'sum_doc_freq': 10184,
                                                 'sum_ttf': 10189},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 0}


In [15]:
from datasets import load_dataset

In [16]:
import datasets
import pandas as pd
import re
import pandas as pd
from datasets import load_from_disk
trainpath = "/opt/ml/data/train_dataset"
testpath = '/opt/ml/data/test_dataset'
datasets = load_from_disk(testpath)
datasets

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})

In [17]:
query = datasets['validation']['question']
# ground_truth = datasets['validation']['context']
id = datasets['validation']['id']
len(query)

600

In [18]:
id[0]

'mrc-1-000653'

In [19]:
correct = 0
total = []
for i, q in enumerate(query):
    q = q.replace('~','-')
    res = es.search(index=INDEX_NAME, q=q, size=20)
    # print('query : ', q)
    # print('id : ', id[i])
    # print(res)
    tmp = {
        'id':id[i],
        # 'text1':res['hits']['hits'][0]['_source']['text'],
        # 'score1':res['hits']['hits'][0]['_score'],
        # 'text2':res['hits']['hits'][1]['_source']['text'],
        # 'score2':res['hits']['hits'][1]['_score'],
        # 'text3':res['hits']['hits'][2]['_source']['text'],
        # 'score3':res['hits']['hits'][2]['_score'],
    }
    tmp2 = {f'text{j+1}':res['hits']['hits'][j]['_source']['text'] for j in range(20)}
    tmp3 = {f'score{j+1}':res['hits']['hits'][j]['_score'] for j in range(20)}
    tmp.update(tmp2)
    tmp.update(tmp3)
    
    total.append(tmp)
    
    # context = res['hits']['hits']
    # print(q)
    # print(context)
    # print('='*100)


# print('% : ', correct/len(query))

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [20]:
import pandas as pd
df = pd.DataFrame(total)
df


,id,text1,text2,text3,text4,text5,text6,text7,text8,text9,...,score11,score12,score13,score14,score15,score16,score17,score18,score19,score20
0,mrc-1-000653,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,글리제 581 g는 천칭자리 방향으로 지구로부터 약 20.5광년 떨어진 적색 왜성 ...,"지구 근접 소행성(地球近接小行星)은 지구 궤도와 교차하는 궤도를 가진 소행성으로, ...","지구 근접 소행성(地球近接小行星)은 지구 궤도와 교차하는 궤도를 가진 소행성으로, ...",서브지구(sub-Earth)는 지구와 금성보다 질량이 확연하게 작은 행성이다 태양계...,페가수스자리 51 b가 발견되기 이전 폴란드 천문학자 알렉산데르 볼시찬이 이미 펄서...,"HD 40307 b의 질량은 적어도 지구의 4.2배 수준인데, 기존 가스 행성에 비...",이 정의는 엄밀해야 할 정의가 모호성을 띄고 있다는 점에서 비판을 많이 받았다. 천...,"이 별은 여러 면에서 태양과 비슷하나, 질량은 태양보다 6퍼센트 더 크며 무거운 원...",...,15.195357,15.061412,15.045091,15.005158,14.952311,14.926474,14.923213,14.920198,14.846245,14.821501
1,mrc-1-001113,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...,SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌...,2014년 1월 10일 KBO 리그 LG 트윈스와 계약하며 2013년 이후 방출된 ...,1955년 8월 1일에 한인 위주로 구성된 <길림성축구팀>(지린축구단)이 창단했다....,에드미우송은 1988년 20세의 나이로 포르투갈에 와서 CD 나시오날에 입단했다. ...,2012년 1월 18일 카림 가르시아의 대체 외국인 선수로 한화 이글스에 입단했다....,2002년 FIFA 월드컵이 끝나고 브라질의 감독으로 선임되었다. 재임 초기에 열린...,2018년 FIFA월드컵 개최를 위해 제출된 팀 휘프의 새로운 경기장은 독일 건축가...,내셔널리그 2010(대한생명의 후원을 받아 대한생명 내셔널리그 2010으로 치름) ...,...,13.457867,13.457867,13.320274,13.215060,12.759902,12.619909,12.522187,12.442860,12.232193,12.095229
2,mrc-0-002191,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ...",돌푸스는 니더외스터라이히주의 텍싱탈에서 싱글맘이자 신실한 가톨릭교인인 어머니 밑에서...,; 디오의 성 : 1부의 무대. 흡혈귀 디오가 근거지로 하고 있던 곳이다. 배경에는...,"부르군트의 콘라트 3세(Conrad III/Konrad III, 922년/925년 ...","650년 섬주(陝州) 협석(硤石)에서 요의(姚懿, 요선의姚善意)의 아들로 태어났다....",1928년 경상북도 성주에서 출생하여 지난날 한때 경상북도 대구에서 잠시 유아기를 ...,금전신탁의 운용방법은 특정한 특정(特定) 금전신탁과 특정하지 않은 불특정(不特定)금...,바우트웰은 1869년 율리시스 S. 그랜트 대통령으로부터 재무 장관의 임명을 받아들...,"13세기 영국의 법률가 브랙턴(Henry de Bracton)의 주저는 ""영국의 법...",...,14.037574,13.988475,13.988475,13.820453,13.783020,13.774279,13.748519,13.661309,13.569367,13.496354
3,mrc-0-003951,"유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민...",멕시코의 언어는 다양하고 그 중 스페인어가 가장 널리 쓰인다. 멕시코 원주민 언어들...,"농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리포르니아주 북부에서는...","영어는 발음, 어휘, 철자법 등이 영국, 북미, 영어를 사용하는 아프리카 국가, 싱...",국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매기는데 그 부호 수...,국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매기는데 그 부호 수...,"세계농아인연맹(World Federation of the Deaf, WFD)은 농아...",1970년대 이전에 니카라과에는 청각장애인의 커뮤니티가 없었다. 청각장애인들은 서로...,미국의 언어 미국은 연방 정부 차원에서 따로 공용어를 지정하지 않고 있다. 하지만 ...,...,25.176722,25.176722,24.757680,24.757680,24.506810,24.480978,24.331877,24.197138,24.197138,24.071556
4,mrc-1-001272,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...,제1차 인도차이나 전쟁 중에는 프랑스 공군의 깟비 공군기지로 이용되었다. 1954년...,그는 1940년대에 자신의 군사 경력을 시작하여 프랑스 식민지 육군에 입대하였다. ...,이 작전에 관해서는 연합군의 침공 외의 전투는 일어나지 않았다. 일본군은 프랑스 주...,러셀 법정은 1966년 영국의 철학자 버트런드 러셀이 베트남 전쟁에서 일어난 전쟁범...,공항이 폐쇄됨에 따라 현재 나산 공항에 대한 예약된 운항 서비스는 없다. 2015년...,제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프랑스의 설득에 따라 ...,제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프랑스의 설득에 따라 ...,"과거 홍강 삼각주의 중심부에 자리 잡고 있었지만, 과거에는 타이빈성은 3개의 큰 강...",...,9.847786,9.797603,9.714008,9.706580,9.669913,9.618228,9.618228,9.458575,9.437896,9.413426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,mrc-0-002989,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,...","후토 타워의 완공 30주년이 되는 해의 여름, A에서 Z까지의 26개의 차세대 가이...",NEVER(NEVER(ネバー)|네바)는 다이도 카츠미가 이끄는 세계적으로 악명 높은...,"""가면라이더 W FOREVER A to Z/운명의 가이아 메모리""(仮面ライダーW F...","IAS 머신은 40비트 워드를 갖춘 이진 방식의 컴퓨터의 하나로서, 워드 하나에 두...",ML은 범용 프로그래밍 언어의 일종이다. 프로그래밍 언어 분야의 핵심 연구 성과들을...,ML은 범용 프로그래밍 언어의 일종이다. 프로그래밍 언어 분야의 핵심 연구 성과들을...,"가이아 건담(Gaia Gundam, ガイアガンダム)은 TV 애니메이션 <기동전사 건...",FB-DIMM(Fully Buffered DIMM)은 메모리 시스템의 신뢰성과 밀집...,...,15.733595,14.913267,13.971909,13.237483,13.226234,13.084192,13.017657,12.709731,12.615415,12.509698
596,mrc-0-001804,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...,4.19 의거나 3.15 부정선거 당시 학생 시위 진압자에 대한 미온적인 처벌이 논...,4.19 의거나 3.15 부정선거 당시 학생 시위 진압자에 대한 미온적인 처벌이 논...,4.19 관련자에 대한 미온적인 처벌이 문제가 되자 그는 결국 1960년 말부터 민...,4.19 관련자에 대한 미온적인 처벌이 문제가 되자 그는 결국 1960년 말부터 민...,"1951년 장

In [21]:
df.to_csv('../add_context_test_dataset.csv')